In [12]:
from tqdm.notebook import tqdm
import os, sys
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np

sys.path.insert(0, "../src")
sys.path.insert(0, "/mnt/nas/minici/SocialAIGym/src")

from data_component import DataComponent
from information_diffusion_component import BoundedConfidenceDiffusionComponent
from graph_utils import *

num_nodes = 100
avg_deg = 5

# bounded confidence model params
epsilon = 0.2
mu = 0.5

def get_optimal_reward(bc_model, llm_node_id, search_granularity=1e-5):
    message_value_list = []
    reward_list = []
    for message_value in tqdm(np.arange(0., 1., search_granularity), "message value"):
        opinion_shift_tot, num_activated_users, _ = bc_model.propagate_message(message=message_value, node_id=llm_node_id)
        message_value_list.append(message_value)
        reward_list.append(num_activated_users)
    return np.max(reward_list), {'message_opinion': message_value_list, 'reward': reward_list}



for LLM_pos in ("echo-low", "echo-high", "comm-largest", "comm-smallest", "central"):
    for MODULARITY in ("high", "low"):
        for HOMOPHILY in ("high", "low"):
            for NETWORK_OPINION in ('neutral', 'positive', 'negative'):
                if MODULARITY == 'high':
                    modularity = 0.75
                elif MODULARITY == 'low':
                    modularity = 0.25
                elif MODULARITY == 'medium':
                    modularity = 0.5
                else:
                    raise Exception(f"modularity {MODULARITY} not in (high, low, medium)")

                if HOMOPHILY == 'high':
                    homophily = 0.75
                elif HOMOPHILY == 'low':
                    homophily = 0.25
                elif HOMOPHILY == 'medium':
                    homophily = 0.5
                else:
                    raise Exception(f"homophily {HOMOPHILY} not in (high, low, medium)")

                if NETWORK_OPINION == 'negative':
                    alpha = 1
                    beta = 10
                elif NETWORK_OPINION == 'positive':
                    alpha = 10
                    beta = 1
                elif NETWORK_OPINION == 'neutral':
                    alpha = 1
                    beta = 1
                else:
                    raise Exception(f"network_opinion {NETWORK_OPINION} not in (positive, negative, neutral)")

                data = DataComponent(num_nodes, modularity, homophily, avg_deg, alpha=alpha, beta=beta)

                if LLM_pos == "echo-low":
                    llm_node_id = LLM_in_echochamber(data, "low")
                elif LLM_pos == "echo-high":
                    llm_node_id = LLM_in_echochamber(data, "high")
                elif LLM_pos == "comm-largest":
                    llm_node_id = LLM_in_comm(data, "largest")
                elif LLM_pos == "comm-smallest":
                    llm_node_id = LLM_in_comm(data, "smallest")
                elif LLM_pos == "central":
                    llm_node_id = LLM_central(data)
                # 1. Init diffusion model ----------------------------------------
                information_diffusion_model = BoundedConfidenceDiffusionComponent(data_component=data, epsilon=epsilon, mu=mu)
                optimal_reward, _ = get_optimal_reward(bc_model=information_diffusion_model, llm_node_id=llm_node_id, search_granularity=1e-5)
                print(f"MODULARITY: {MODULARITY}; HOMOPHILY: {HOMOPHILY}; NETWORK_OPINION: {NETWORK_OPINION}; LLM_pos: {LLM_pos}")
                print("Optimal reward: ", optimal_reward)


message value:   0%|          | 0/100000 [00:00<?, ?it/s]

MODULARITY: high; HOMOPHILY: high; NETWORK_OPINION: neutral; LLM_pos: echo-low
Optimal reward:  47


message value:   0%|          | 0/100000 [00:00<?, ?it/s]

MODULARITY: high; HOMOPHILY: high; NETWORK_OPINION: positive; LLM_pos: echo-low
Optimal reward:  99


message value:   0%|          | 0/100000 [00:00<?, ?it/s]

...

MODULARITY: low; HOMOPHILY: low; NETWORK_OPINION: neutral; LLM_pos: central
Optimal reward:  47


message value:   0%|          | 0/100000 [00:00<?, ?it/s]

MODULARITY: low; HOMOPHILY: low; NETWORK_OPINION: positive; LLM_pos: central
Optimal reward:  99


message value:   0%|          | 0/100000 [00:00<?, ?it/s]

MODULARITY: low; HOMOPHILY: low; NETWORK_OPINION: negative; LLM_pos: central
Optimal reward:  97
